In [1]:
import pandas
import json

In [2]:
def writeJSONStrToFile(json_str, name):
    jsonFile = open(name + '.json', 'w')
    jsonFile.write(json_str)
    jsonFile.close()

def writeJSONObjToFile(json_obj, name):
    jsonFile = open(name + '.json', 'w')
    jsonFile.write(json.dumps(json_obj))
    jsonFile.close()

In [3]:
# Raw JSON data from XLSX
excel_data_df = pandas.read_excel('Data_Explorer_Dataset.xlsx', sheet_name='Data Explorer Report')
mydict = excel_data_df.to_dict(orient='records')
json_str = excel_data_df.to_json(orient='records')

writeJSONStrToFile(json_str, 'raw_data')

with open('raw_data.json') as f:
    raw_data = list(json.load(f))

In [4]:
# JSON data with no duplicate locations
seen_coords_lat = set()
seen_coords_long = set()
data_no_duplicates = []
for item in raw_data:
    if item['Latitude'] not in seen_coords_lat and item['Longitude'] not in seen_coords_long:
        data_no_duplicates.append(item)
        seen_coords_lat.add(item['Latitude'])
        seen_coords_long.add(item['Longitude'])

writeJSONObjToFile(data_no_duplicates, "data_no_duplicates")

In [5]:
# Prepare JSON data for filtering
data_list_for_filter = []
for item in data_no_duplicates:
    item_organ_list = [organ[1:] for organ in item["OTC Service List"].split(',')]
    item['OTC Service List'] = item_organ_list
    data_list_for_filter.append(item)

writeJSONObjToFile(data_list_for_filter, 'data_list_for_filter')

In [6]:
# Create a map for mapping JSON data to a filter selection
data_map = {}
counter = 0
for item in data_list_for_filter:
    for organ in item['OTC Service List']:
        if organ not in data_map:
            data_map[organ] = []
        data_map[organ].append(counter)
    counter += 1

writeJSONObjToFile(data_map, 'data_map')